#### Import moduldes

In [1]:
!pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import catboost as cb

from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

pd.set_option('display.max_columns', None)

In [3]:
%cd /content/drive/MyDrive/Olymps/zindi/football

/content/drive/MyDrive/Olymps/zindi/football


In [4]:
def preprocess(train, test):
    target = train.pop('Score').replace({'Away win': 0, 'Draw': 1, 'Home Win': 2})
    # temp = pd.concat([train, target], axis=1)
    temp = pd.concat([train, test], axis=0)
    idk = pd.get_dummies(temp['Home Team'])
    temp = pd.concat([temp, idk], axis=1)
    idk = pd.get_dummies(temp['Away Team'])
    temp = pd.concat([temp, idk], axis=1)
    temp = temp.join(idk, rsuffix='_right')

    return temp, target

In [5]:
train = pd.read_csv("Train.csv")
train.set_index('Game_ID', inplace=True)
test = pd.read_csv("Test.csv")
test.set_index('Game_ID', inplace=True)
df, target = preprocess(train, test)
train_ids, test_ids = train.index, test.index
train = df.loc[train_ids]
test = df.loc[test_ids]
test

,Date,Season,Match_ID,Home Team,Away Team,Andromeda,Antennae,Backward,Butterfly,Cartwheel,Cigar,Circinus,Coma Pinwheel,Comet,Cosmos Redshift 7,Eye of Sauron,Fireworks,Malin 1,Medusa Merger,Milky Way,Needle,Sculptor,Sombrero,Sunflower,Tadpole,Triangulum,Whirlpool,Andromeda,Antennae,Backward,Butterfly,Cartwheel,Cigar,Circinus,Coma Pinwheel,Comet,Cosmos Redshift 7,Eye of Sauron,Fireworks,Malin 1,Medusa Merger,Milky Way,Needle,Sculptor,Sombrero,Sunflower,Tadpole,Triangulum,Whirlpool,Andromeda_right,Antennae_right,Backward_right,Butterfly_right,Cartwheel_right,Cigar_right,Circinus_right,Coma Pinwheel_right,Comet_right,Cosmos Redshift 7_right,Eye of Sauron_right,Fireworks_right,Malin 1_right,Medusa Merger_right,Milky Way_right,Needle_right,Sculptor_right,Sombrero_right,Sunflower_right,Tadpole_right,Triangulum_right,Whirlpool_right
Game_ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
ID_8518U587,2018-12-08,3,102.0,Antennae,Cosmos Redshift 7,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
ID_H49BIKG7,2018-09-11,3,18.0,Cosmos Redshift 7,Tadpole,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
ID_PO6SP4VA,2018-09-25,3,32.0,Cosmos Redshift 7,Medusa Merger,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
ID_MZRCNBAQ,2018-10-02,3,40.0,Cartwheel,Cosmos Redshift 7,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
ID_CV9VOLIU,2018-12-20,3,123.0,Cosmos Redshift 7,Needle,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ID_QKQ00O16,2018-12-08,3,300.0,Malin 1,Needle,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
ID_172V2IXW,2018-12-11,3,300.0,Tadpole,Needle,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
ID_2ZK5TE39,2018-12-22,3,300.0,Circinus,Malin 1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0


In [6]:
game_stats = pd.read_csv("game_statistics.csv")
depr = ['Goals_scored', 'Goals_conceded', 'Action']
for d in depr:
    game_stats.pop(d)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (19,20,24) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [8]:
game_stats

,Game_ID,Player_ID,id,X,Y,Team,Half,Manager,Opposition_Team,Shots,SoT,Accurate passes,Inaccurate passes,Passes,Start_minutes,End_minutes,next_player,next_action,next_x,next_y,event_id,next_team,next_event_id,xt_value
0,ID_ZZKYYPVY,Player_O7HO1NOB,1.0,52.5,34.0,Circinus,1st half,Ryoichi,Backward,0.0,0.0,1.0,0.0,1.0,0.03,0.37,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ID_ZZKYYPVY,Player_O7HO1NOB,2.0,52.5,34.0,Circinus,1st half,Ryoichi,Backward,0.0,0.0,0.0,0.0,0.0,0.03,0.37,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ID_ZZKYYPVY,Player_J0V7B0YE,3.0,37.0,32.7,Circinus,1st half,Ryoichi,Backward,0.0,0.0,0.0,0.0,0.0,0.05,0.38,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ID_ZZKYYPVY,Player_J0V7B0YE,4.0,39.5,33.0,Circinus,1st half,Ryoichi,Backward,0.0,0.0,0.0,0.0,0.0,0.07,0.40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ID_ZZKYYPVY,Player_J0V7B0YE,5.0,39.5,33.0,Circinus,1st half,Ryoichi,Backward,0.0,0.0,0.0,1.0,1.0,0.07,0.40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2428096,ID_00EFNL7L,Player_30LIPYOT,3454.0,15.3,21.3,Milky Way,2nd half,Masashi,Butterfly,0.0,0.0,0.0,0.0,0.0,97.20,97.53,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2428097,ID_00EFNL7L,Player_3B44A2GJ,3455.0,88.6,48.7,Butterfly,2nd half,Dante,Milky Way,0.0,0.0,0.0,0.0,0.0,97.22,97.55,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2428098,ID_00EFNL7L,Player_3B44A2GJ,3456.0,88.6,48.7,Butterfly,2nd half,Dante,Milky Way,0.0,0.0,0.0,0.0,0.0,97.22,97.55,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2428099,ID_00EFNL7L,Player_DICH31J8,3457.0,18.1,19.7,Milky Way,2nd half,Masashi,Butterfly,0.0,0.0,0.0,1.0,1.0,97.77,98.10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
gs_old = game_stats[:]

In [10]:
# depr = ['Goals_scored', 'Goals_conceded', 'Action']
# for d in depr:
#     gs_old.pop(d)

In [8]:
gs_old

,Game_ID,Player_ID,id,X,Y,Team,Half,Manager,Opposition_Team,Shots,SoT,Accurate passes,Inaccurate passes,Passes,Start_minutes,End_minutes,next_player,next_action,next_x,next_y,event_id,next_team,next_event_id,xt_value
0,ID_ZZKYYPVY,Player_O7HO1NOB,1.0,52.5,34.0,Circinus,1st half,Ryoichi,Backward,0.0,0.0,1.0,0.0,1.0,0.03,0.37,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ID_ZZKYYPVY,Player_O7HO1NOB,2.0,52.5,34.0,Circinus,1st half,Ryoichi,Backward,0.0,0.0,0.0,0.0,0.0,0.03,0.37,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ID_ZZKYYPVY,Player_J0V7B0YE,3.0,37.0,32.7,Circinus,1st half,Ryoichi,Backward,0.0,0.0,0.0,0.0,0.0,0.05,0.38,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ID_ZZKYYPVY,Player_J0V7B0YE,4.0,39.5,33.0,Circinus,1st half,Ryoichi,Backward,0.0,0.0,0.0,0.0,0.0,0.07,0.40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ID_ZZKYYPVY,Player_J0V7B0YE,5.0,39.5,33.0,Circinus,1st half,Ryoichi,Backward,0.0,0.0,0.0,1.0,1.0,0.07,0.40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2428096,ID_00EFNL7L,Player_30LIPYOT,3454.0,15.3,21.3,Milky Way,2nd half,Masashi,Butterfly,0.0,0.0,0.0,0.0,0.0,97.20,97.53,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2428097,ID_00EFNL7L,Player_3B44A2GJ,3455.0,88.6,48.7,Butterfly,2nd half,Dante,Milky Way,0.0,0.0,0.0,0.0,0.0,97.22,97.55,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2428098,ID_00EFNL7L,Player_3B44A2GJ,3456.0,88.6,48.7,Butterfly,2nd half,Dante,Milky Way,0.0,0.0,0.0,0.0,0.0,97.22,97.55,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2428099,ID_00EFNL7L,Player_DICH31J8,3457.0,18.1,19.7,Milky Way,2nd half,Masashi,Butterfly,0.0,0.0,0.0,1.0,1.0,97.77,98.10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
gs_old['next_action'].value_counts()

Positional attacks             283262
Accurate passes                157172
Counter-attacks                 44912
Inaccurate passes               36037
Lost balls                      30818
Challenges lost                 30529
Challenges won                  25625
Picking-ups                     24341
Interceptions                   21583
Passes into the penalty box     15417
Corner attacks                  11881
Air challenges won               9275
Air challenges lost              9274
Opp half pick-ups                8857
Free-kick attacks                8025
Dribbling                        6866
Successful tackles               6658
Opp half lost balls              5852
Fouls                            5579
Successful dribbles              4905
Unsuccessful tackles             4905
Opp half interceptions           4180
Shots                            4065
Throw-in attacks                 4008
Inaccurate crosses               3894
Bad ball control                 3796
Unsuccessful

In [8]:
gs_old.loc[gs_old['next_action'] == 'Goals', 'chance'] = 1
gs_old.loc[~(gs_old['next_action'] == 'Goals'), 'chance'] = 0
gs_old

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1684: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = infer_fill_value(value)
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,Game_ID,Player_ID,id,X,Y,Team,Half,Manager,Opposition_Team,Shots,SoT,Accurate passes,Inaccurate passes,Passes,Start_minutes,End_minutes,next_player,next_action,next_x,next_y,event_id,next_team,next_event_id,xt_value,chance
0,ID_ZZKYYPVY,Player_O7HO1NOB,1.0,52.5,34.0,Circinus,1st half,Ryoichi,Backward,0.0,0.0,1.0,0.0,1.0,0.03,0.37,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
1,ID_ZZKYYPVY,Player_O7HO1NOB,2.0,52.5,34.0,Circinus,1st half,Ryoichi,Backward,0.0,0.0,0.0,0.0,0.0,0.03,0.37,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
2,ID_ZZKYYPVY,Player_J0V7B0YE,3.0,37.0,32.7,Circinus,1st half,Ryoichi,Backward,0.0,0.0,0.0,0.0,0.0,0.05,0.38,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
3,ID_ZZKYYPVY,Player_J0V7B0YE,4.0,39.5,33.0,Circinus,1st half,Ryoichi,Backward,0.0,0.0,0.0,0.0,0.0,0.07,0.40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
4,ID_ZZKYYPVY,Player_J0V7B0YE,5.0,39.5,33.0,Circinus,1st half,Ryoichi,Backward,0.0,0.0,0.0,1.0,1.0,0.07,0.40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2428096,ID_00EFNL7L,Player_30LIPYOT,3454.0,15.3,21.3,Milky Way,2nd half,Masashi,Butterfly,0.0,0.0,0.0,0.0,0.0,97.20,97.53,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
2428097,ID_00EFNL7L,Player_3B44A2GJ,3455.0,88.6,48.7,Butterfly,2nd half,Dante,Milky Way,0.0,0.0,0.0,0.0,0.0,97.22,97.55,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
2428098,ID_00EFNL7L,Player_3B44A2GJ,3456.0,88.6,48.7,Butterfly,2nd half,Dante,Milky Way,0.0,0.0,0.0,0.0,0.0,97.22,97.55,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
2428099,ID_00EFNL7L,Player_DICH31J8,3457.0,18.1,19.7,Milky Way,2nd half,Masashi,Butterfly,0.0,0.0,0.0,1.0,1.0,97.77,98.10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0


In [9]:
gs_old.loc[(gs_old['X'] >= 88) & (gs_old['Y'] <  38) & (gs_old['Y'] > 22), 'attacks'] = 1
gs_old.loc[~(gs_old['X'] >= 88) & (gs_old['Y'] <  38) & (gs_old['Y'] > 22), 'attacks'] = 0
gs_old[gs_old['attacks'] == 1]

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1684: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = infer_fill_value(value)
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,Game_ID,Player_ID,id,X,Y,Team,Half,Manager,Opposition_Team,Shots,SoT,Accurate passes,Inaccurate passes,Passes,Start_minutes,End_minutes,next_player,next_action,next_x,next_y,event_id,next_team,next_event_id,xt_value,chance,attacks
57,ID_ZZKYYPVY,Player_OOO2DN3U,59.0,94.9,35.4,Circinus,1st half,Ryoichi,Backward,0.0,0.0,0.0,0.0,0.0,0.78,1.12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0
59,ID_ZZKYYPVY,Player_OOO2DN3U,61.0,94.9,35.4,Circinus,1st half,Ryoichi,Backward,0.0,0.0,0.0,0.0,0.0,0.78,1.12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0
60,ID_ZZKYYPVY,Player_OOO2DN3U,62.0,94.9,35.4,Circinus,1st half,Ryoichi,Backward,0.0,0.0,0.0,0.0,0.0,0.78,1.12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0
72,ID_ZZKYYPVY,Player_AX9GZJU4,74.0,90.0,35.6,Backward,1st half,Karatachi,Circinus,0.0,0.0,0.0,0.0,0.0,1.02,1.35,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0
76,ID_ZZKYYPVY,Player_AX9GZJU4,78.0,96.7,34.2,Backward,1st half,Karatachi,Circinus,0.0,0.0,0.0,0.0,0.0,1.03,1.37,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2427835,ID_00EFNL7L,Player_QP5O0QD4,3193.0,96.8,34.0,Butterfly,2nd half,Dante,Milky Way,1.0,0.0,0.0,0.0,0.0,88.47,88.80,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0
2427846,ID_00EFNL7L,Player_033SPDRN,3204.0,97.7,34.8,Butterfly,2nd half,Dante,Milky Way,0.0,0.0,0.0,0.0,0.0,89.03,89.37,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0
2427877,ID_00EFNL7L,Player_ZQ4VOP93,3235.0,98.3,27.6,Butterfly,2nd half,Dante,Milky Way,0.0,0.0,0.0,0.0,0.0,89.45,89.78,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0
2428008,ID_00EFNL7L,Player_10WKX99Y,3366.0,96.4,33.9,Milky Way,2nd half,Masashi,Butterfly,0.0,0.0,0.0,0.0,0.0,94.17,95.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0


In [40]:
gs_old

,Game_ID,Player_ID,id,X,Y,Team,Half,Manager,Opposition_Team,Shots,SoT,Accurate passes,Inaccurate passes,Passes,Start_minutes,End_minutes,next_player,next_action,next_x,next_y,event_id,next_team,next_event_id,xt_value,chance,attacks
0,ID_ZZKYYPVY,Player_O7HO1NOB,1.0,52.5,34.0,Circinus,1st half,Ryoichi,Backward,0.0,0.0,1.0,0.0,1.0,0.03,0.37,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0
1,ID_ZZKYYPVY,Player_O7HO1NOB,2.0,52.5,34.0,Circinus,1st half,Ryoichi,Backward,0.0,0.0,0.0,0.0,0.0,0.03,0.37,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0
2,ID_ZZKYYPVY,Player_J0V7B0YE,3.0,37.0,32.7,Circinus,1st half,Ryoichi,Backward,0.0,0.0,0.0,0.0,0.0,0.05,0.38,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0
3,ID_ZZKYYPVY,Player_J0V7B0YE,4.0,39.5,33.0,Circinus,1st half,Ryoichi,Backward,0.0,0.0,0.0,0.0,0.0,0.07,0.40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0
4,ID_ZZKYYPVY,Player_J0V7B0YE,5.0,39.5,33.0,Circinus,1st half,Ryoichi,Backward,0.0,0.0,0.0,1.0,1.0,0.07,0.40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2428096,ID_00EFNL7L,Player_30LIPYOT,3454.0,15.3,21.3,Milky Way,2nd half,Masashi,Butterfly,0.0,0.0,0.0,0.0,0.0,97.20,97.53,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
2428097,ID_00EFNL7L,Player_3B44A2GJ,3455.0,88.6,48.7,Butterfly,2nd half,Dante,Milky Way,0.0,0.0,0.0,0.0,0.0,97.22,97.55,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
2428098,ID_00EFNL7L,Player_3B44A2GJ,3456.0,88.6,48.7,Butterfly,2nd half,Dante,Milky Way,0.0,0.0,0.0,0.0,0.0,97.22,97.55,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
2428099,ID_00EFNL7L,Player_DICH31J8,3457.0,18.1,19.7,Milky Way,2nd half,Masashi,Butterfly,0.0,0.0,0.0,1.0,1.0,97.77,98.10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN


In [13]:
from tqdm import tqdm

def feats(temp):
    # print('agg', game_stats.columns.tolist()[n:])
    lst = ['Shots', 'SoT', 'Accurate passes', 'Inaccurate passes', 'Passes', 'attacks', 'chance']
    for team in tqdm(temp['Team'].unique()):
        for c in lst:
            # print(f'{c}_{team}')
            temp[f'{c}_{team}'] = temp.loc[temp['Team'] == team, c]
    temp.fillna(0, inplace=True)
    return temp

def feats2(temp):
    gs = pd.DataFrame()
    n = temp.columns.get_loc('Shots_Circinus') 
    for c in tqdm(temp.columns.tolist()[n:]):
        print(c)
        # print(game_stats[c].value_counts())
        gs[c] = temp.groupby('Game_ID')[c].sum()
    return gs

In [ ]:
for i, c in enumerate(gs_old.columns.tolist()):
    print(i, c)

In [ ]:
gs_old = feats(gs_old)
gs_old

In [14]:
gs_new = feats2(gs_old)
gs_new

  0%|          | 0/154 [00:00<?, ?it/s]

Shots_Circinus


  1%|          | 1/154 [00:00<00:57,  2.66it/s]

SoT_Circinus


  1%|▏         | 2/154 [00:00<00:54,  2.81it/s]

...

 99%|█████████▊| 152/154 [00:32<00:00,  5.63it/s]

Passes_Comet
attacks_Comet


100%|██████████| 154/154 [00:32<00:00,  4.71it/s]


chance_Comet


,Shots_Circinus,SoT_Circinus,Accurate passes_Circinus,Inaccurate passes_Circinus,Passes_Circinus,attacks_Circinus,chance_Circinus,Shots_Backward,SoT_Backward,Accurate passes_Backward,Inaccurate passes_Backward,Passes_Backward,attacks_Backward,chance_Backward,Shots_Sunflower,SoT_Sunflower,Accurate passes_Sunflower,Inaccurate passes_Sunflower,Passes_Sunflower,attacks_Sunflower,chance_Sunflower,Shots_Coma Pinwheel,SoT_Coma Pinwheel,Accurate passes_Coma Pinwheel,Inaccurate passes_Coma Pinwheel,Passes_Coma Pinwheel,attacks_Coma Pinwheel,chance_Coma Pinwheel,Shots_Eye of Sauron,SoT_Eye of Sauron,Accurate passes_Eye of Sauron,Inaccurate passes_Eye of Sauron,Passes_Eye of Sauron,attacks_Eye of Sauron,chance_Eye of Sauron,Shots_Butterfly,SoT_Butterfly,Accurate passes_Butterfly,Inaccurate passes_Butterfly,Passes_Butterfly,attacks_Butterfly,chance_Butterfly,Shots_Sombrero,SoT_Sombrero,Accurate passes_Sombrero,Inaccurate passes_Sombrero,Passes_Sombrero,attacks_Sombrero,chance_Sombrero,Shots_Milky Way,SoT_Milky Way,Accurate passes_Milky Way,Inaccurate passes_Milky Way,Passes_Milky Way,attacks_Milky Way,chance_Milky Way,Shots_Antennae,SoT_Antennae,Accurate passes_Antennae,Inaccurate passes_Antennae,Passes_Antennae,attacks_Antennae,chance_Antennae,Shots_Whirlpool,SoT_Whirlpool,Accurate passes_Whirlpool,Inaccurate passes_Whirlpool,Passes_Whirlpool,attacks_Whirlpool,chance_Whirlpool,Shots_Malin 1,SoT_Malin 1,Accurate passes_Malin 1,Inaccurate passes_Malin 1,Passes_Malin 1,attacks_Malin 1,chance_Malin 1,Shots_Tadpole,SoT_Tadpole,Accurate passes_Tadpole,Inaccurate passes_Tadpole,Passes_Tadpole,attacks_Tadpole,chance_Tadpole,Shots_Cosmos Redshift 7,SoT_Cosmos Redshift 7,Accurate passes_Cosmos Redshift 7,Inaccurate passes_Cosmos Redshift 7,Passes_Cosmos Redshift 7,attacks_Cosmos Redshift 7,chance_Cosmos Redshift 7,Shots_Cigar,SoT_Cigar,Accurate passes_Cigar,Inaccurate passes_Cigar,Passes_Cigar,attacks_Cigar,chance_Cigar,Shots_Cartwheel,SoT_Cartwheel,Accurate passes_Cartwheel,Inaccurate passes_Cartwheel,Passes_Cartwheel,attacks_Cartwheel,chance_Cartwheel,Shots_Fireworks,SoT_Fireworks,Accurate passes_Fireworks,Inaccurate passes_Fireworks,Passes_Fireworks,attacks_Fireworks,chance_Fireworks,Shots_Needle,SoT_Needle,Accurate passes_Needle,Inaccurate passes_Needle,Passes_Needle,attacks_Needle,chance_Needle,Shots_Sculptor,SoT_Sculptor,Accurate passes_Sculptor,Inaccurate passes_Sculptor,Passes_Sculptor,attacks_Sculptor,chance_Sculptor,Shots_Triangulum,SoT_Triangulum,Accurate passes_Triangulum,Inaccurate passes_Triangulum,Passes_Triangulum,attacks_Triangulum,chance_Triangulum,Shots_Andromeda,SoT_Andromeda,Accurate passes_Andromeda,Inaccurate passes_Andromeda,Passes_Andromeda,attacks_Andromeda,chance_Andromeda,Shots_Medusa Merger,SoT_Medusa Merger,Accurate passes_Medusa Merger,Inaccurate passes_Medusa Merger,Passes_Medusa Merger,attacks_Medusa Merger,chance_Medusa Merger,Shots_Comet,SoT_Comet,Accurate passes_Comet,Inaccurate passes_Comet,Passes_Comet,attacks_Comet,chance_Comet
Game_ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
ID_00EFNL7L,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,19.0,7.0,327.0,83.0,410.0,58.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,263.0,86.0,349.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ID_00J124MI,0.0,0.0,0.0,0.0,0.0,0.0,0.0,18.0,6.0,259.0,71.0,330.0,32.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.

In [15]:
train

,Date,Season,Match_ID,Home Team,Away Team,Andromeda,Antennae,Backward,Butterfly,Cartwheel,Cigar,Circinus,Coma Pinwheel,Comet,Cosmos Redshift 7,Eye of Sauron,Fireworks,Malin 1,Medusa Merger,Milky Way,Needle,Sculptor,Sombrero,Sunflower,Tadpole,Triangulum,Whirlpool,Andromeda,Antennae,Backward,Butterfly,Cartwheel,Cigar,Circinus,Coma Pinwheel,Comet,Cosmos Redshift 7,Eye of Sauron,Fireworks,Malin 1,Medusa Merger,Milky Way,Needle,Sculptor,Sombrero,Sunflower,Tadpole,Triangulum,Whirlpool,Andromeda_right,Antennae_right,Backward_right,Butterfly_right,Cartwheel_right,Cigar_right,Circinus_right,Coma Pinwheel_right,Comet_right,Cosmos Redshift 7_right,Eye of Sauron_right,Fireworks_right,Malin 1_right,Medusa Merger_right,Milky Way_right,Needle_right,Sculptor_right,Sombrero_right,Sunflower_right,Tadpole_right,Triangulum_right,Whirlpool_right
Game_ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
ID_KAG4KAE9,2017-02-15,1,1.0,Antennae,Andromeda,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
ID_T7H1XN5Z,2016-09-14,1,12.0,Andromeda,Antennae,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
ID_O1P9FGN4,2016-12-12,1,13.0,Andromeda,Butterfly,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
ID_TR5M4KQV,2017-01-25,1,14.0,Andromeda,Cigar,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
ID_EOIECRE2,2016-08-24,1,15.0,Andromeda,Circinus,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ID_PREVA6HZ,2016-08-28,1,59.0,Sculptor,Medusa Merger,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
ID_9DJ219JY,2017-02-19,1,151.0,Medusa Merger,Sculptor,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
ID_A1PN23J6,2016-10-23,1,154.0,Medusa Merger,Sombrero,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0


In [ ]:
pd.concat([train, test], axis=0)['Date'].value_counts()

2018-06-02    7
2019-05-14    7
2017-10-28    6
2018-05-01    6
2018-08-25    6
             ..
2018-04-25    1
2017-01-31    1
2017-08-17    1
2017-02-14    1
2019-05-06    1
Name: Date, Length: 269, dtype: int64

In [16]:
gs_old.set_index('Game_ID', inplace=True)
gs_old

,Player_ID,id,X,Y,Team,Half,Manager,Opposition_Team,Shots,SoT,Accurate passes,Inaccurate passes,Passes,Start_minutes,End_minutes,next_player,next_action,next_x,next_y,event_id,next_team,next_event_id,xt_value,chance,attacks,Shots_Circinus,SoT_Circinus,Accurate passes_Circinus,Inaccurate passes_Circinus,Passes_Circinus,attacks_Circinus,chance_Circinus,Shots_Backward,SoT_Backward,Accurate passes_Backward,Inaccurate passes_Backward,Passes_Backward,attacks_Backward,chance_Backward,Shots_Sunflower,SoT_Sunflower,Accurate passes_Sunflower,Inaccurate passes_Sunflower,Passes_Sunflower,attacks_Sunflower,chance_Sunflower,Shots_Coma Pinwheel,SoT_Coma Pinwheel,Accurate passes_Coma Pinwheel,Inaccurate passes_Coma Pinwheel,Passes_Coma Pinwheel,attacks_Coma Pinwheel,chance_Coma Pinwheel,Shots_Eye of Sauron,SoT_Eye of Sauron,Accurate passes_Eye of Sauron,Inaccurate passes_Eye of Sauron,Passes_Eye of Sauron,attacks_Eye of Sauron,chance_Eye of Sauron,Shots_Butterfly,SoT_Butterfly,Accurate passes_Butterfly,Inaccurate passes_Butterfly,Passes_Butterfly,attacks_Butterfly,chance_Butterfly,Shots_Sombrero,SoT_Sombrero,Accurate passes_Sombrero,Inaccurate passes_Sombrero,Passes_Sombrero,attacks_Sombrero,chance_Sombrero,Shots_Milky Way,SoT_Milky Way,Accurate passes_Milky Way,Inaccurate passes_Milky Way,Passes_Milky Way,attacks_Milky Way,chance_Milky Way,Shots_Antennae,SoT_Antennae,Accurate passes_Antennae,Inaccurate passes_Antennae,Passes_Antennae,attacks_Antennae,chance_Antennae,Shots_Whirlpool,SoT_Whirlpool,Accurate passes_Whirlpool,Inaccurate passes_Whirlpool,Passes_Whirlpool,attacks_Whirlpool,chance_Whirlpool,Shots_Malin 1,SoT_Malin 1,Accurate passes_Malin 1,Inaccurate passes_Malin 1,Passes_Malin 1,attacks_Malin 1,chance_Malin 1,Shots_Tadpole,SoT_Tadpole,Accurate passes_Tadpole,Inaccurate passes_Tadpole,Passes_Tadpole,attacks_Tadpole,chance_Tadpole,Shots_Cosmos Redshift 7,SoT_Cosmos Redshift 7,Accurate passes_Cosmos Redshift 7,Inaccurate passes_Cosmos Redshift 7,Passes_Cosmos Redshift 7,attacks_Cosmos Redshift 7,chance_Cosmos Redshift 7,Shots_Cigar,SoT_Cigar,Accurate passes_Cigar,Inaccurate passes_Cigar,Passes_Cigar,attacks_Cigar,chance_Cigar,Shots_Cartwheel,SoT_Cartwheel,Accurate passes_Cartwheel,Inaccurate passes_Cartwheel,Passes_Cartwheel,attacks_Cartwheel,chance_Cartwheel,Shots_Fireworks,SoT_Fireworks,Accurate passes_Fireworks,Inaccurate passes_Fireworks,Passes_Fireworks,attacks_Fireworks,chance_Fireworks,Shots_Needle,SoT_Needle,Accurate passes_Needle,Inaccurate passes_Needle,Passes_Needle,attacks_Needle,chance_Needle,Shots_Sculptor,SoT_Sculptor,Accurate passes_Sculptor,Inaccurate passes_Sculptor,Passes_Sculptor,attacks_Sculptor,chance_Sculptor,Shots_Triangulum,SoT_Triangulum,Accurate passes_Triangulum,Inaccurate passes_Triangulum,Passes_Triangulum,attacks_Triangulum,chance_Triangulum,Shots_Andromeda,SoT_Andromeda,Accurate passes_Andromeda,Inaccurate passes_Andromeda,Passes_Andromeda,attacks_Andromeda,chance_Andromeda,Shots_Medusa Merger,SoT_Medusa Merger,Accurate passes_Medusa Merger,Inaccurate passes_Medusa Merger,Passes_Medusa Merger,attacks_Medusa Merger,chance_Medusa Merger,Shots_Comet,SoT_Comet,Accurate passes_Comet,Inaccurate passes_Comet,Passes_Comet,attacks_Comet,chance_Comet
Game_ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
ID_ZZKYYPVY,Player_O7HO1NOB,1.0,52.5,34.0,Circinus,1st half,Ryoichi,Backward,0.0,0.0,1.0,0.0,1.0,0.03,0.37,0,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0

In [17]:
# df = pd.concat([train, test], axis=0)
# df = df.join(pd.get_dummies(df['Date']))
train = df.loc[train_ids]
test = df.loc[test_ids]

lst = ['Shots', 'SoT', 'Accurate passes', 'Inaccurate passes', 'Passes', 'attacks', 'chance']

for place in ['Home', 'Away']:
    for c in lst:
        train[f'{place}_{c}'] = np.nan

    for id in train.index.tolist():
        for c in lst:
            print(f'{c}_{train.loc[id][f"{place} Team"]}')
            # print(id)
            train.loc[id, f'{place}_{c}'] = gs_new.loc[id][f'{c}_{train.loc[id][f"{place} Team"]}']
            # break

    for c in lst:
        test[f'{place}_{c}'] = np.nan

    for id in test.index.tolist():
        for c in lst:
            # print(id)
            test.loc[id, f'{place}_{c}'] = gs_new.loc[id][f'{c}_{test.loc[id][f"{place} Team"]}']
            # break

Выходные данные были обрезаны до нескольких последних строк (5000).
attacks_Backward
chance_Backward
Shots_Backward
SoT_Backward
Accurate passes_Backward
Inaccurate passes_Backward
Passes_Backward
attacks_Backward
chance_Backward
Shots_Backward
SoT_Backward
Accurate passes_Backward
Inaccurate passes_Backward
Passes_Backward
attacks_Backward
chance_Backward
Shots_Backward
SoT_Backward
Accurate passes_Backward
Inaccurate passes_Backward
Passes_Backward
attacks_Backward
chance_Backward
Shots_Cartwheel
SoT_Cartwheel
Accurate passes_Cartwheel
Inaccurate passes_Cartwheel
Passes_Cartwheel
attacks_Cartwheel
chance_Cartwheel
Shots_Circinus
SoT_Circinus
Accurate passes_Circinus
Inaccurate passes_Circinus
Passes_Circinus
attacks_Circinus
chance_Circinus
Shots_Sombrero
SoT_Sombrero
Accurate passes_Sombrero
Inaccurate passes_Sombrero
Passes_Sombrero
attacks_Sombrero
chance_Sombrero
Shots_Eye of Sauron
SoT_Eye of Sauron
Accurate passes_Eye of Sauron
Inaccurate passes_Eye of Sauron
Passes_Eye of Saur

In [18]:
df

,Date,Season,Match_ID,Home Team,Away Team,Andromeda,Antennae,Backward,Butterfly,Cartwheel,Cigar,Circinus,Coma Pinwheel,Comet,Cosmos Redshift 7,Eye of Sauron,Fireworks,Malin 1,Medusa Merger,Milky Way,Needle,Sculptor,Sombrero,Sunflower,Tadpole,Triangulum,Whirlpool,Andromeda,Antennae,Backward,Butterfly,Cartwheel,Cigar,Circinus,Coma Pinwheel,Comet,Cosmos Redshift 7,Eye of Sauron,Fireworks,Malin 1,Medusa Merger,Milky Way,Needle,Sculptor,Sombrero,Sunflower,Tadpole,Triangulum,Whirlpool,Andromeda_right,Antennae_right,Backward_right,Butterfly_right,Cartwheel_right,Cigar_right,Circinus_right,Coma Pinwheel_right,Comet_right,Cosmos Redshift 7_right,Eye of Sauron_right,Fireworks_right,Malin 1_right,Medusa Merger_right,Milky Way_right,Needle_right,Sculptor_right,Sombrero_right,Sunflower_right,Tadpole_right,Triangulum_right,Whirlpool_right
Game_ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
ID_KAG4KAE9,2017-02-15,1,1.0,Antennae,Andromeda,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
ID_T7H1XN5Z,2016-09-14,1,12.0,Andromeda,Antennae,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
ID_O1P9FGN4,2016-12-12,1,13.0,Andromeda,Butterfly,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
ID_TR5M4KQV,2017-01-25,1,14.0,Andromeda,Cigar,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
ID_EOIECRE2,2016-08-24,1,15.0,Andromeda,Circinus,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ID_QKQ00O16,2018-12-08,3,300.0,Malin 1,Needle,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
ID_172V2IXW,2018-12-11,3,300.0,Tadpole,Needle,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
ID_2ZK5TE39,2018-12-22,3,300.0,Circinus,Malin 1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0


In [19]:
test['Home_chance'].value_counts()

0.0    113
1.0     61
2.0     41
3.0     11
4.0      7
5.0      1
Name: Home_chance, dtype: int64

In [20]:
train

,Date,Season,Match_ID,Home Team,Away Team,Andromeda,Antennae,Backward,Butterfly,Cartwheel,Cigar,Circinus,Coma Pinwheel,Comet,Cosmos Redshift 7,Eye of Sauron,Fireworks,Malin 1,Medusa Merger,Milky Way,Needle,Sculptor,Sombrero,Sunflower,Tadpole,Triangulum,Whirlpool,Andromeda,Antennae,Backward,Butterfly,Cartwheel,Cigar,Circinus,Coma Pinwheel,Comet,Cosmos Redshift 7,Eye of Sauron,Fireworks,Malin 1,Medusa Merger,Milky Way,Needle,Sculptor,Sombrero,Sunflower,Tadpole,Triangulum,Whirlpool,Andromeda_right,Antennae_right,Backward_right,Butterfly_right,Cartwheel_right,Cigar_right,Circinus_right,Coma Pinwheel_right,Comet_right,Cosmos Redshift 7_right,Eye of Sauron_right,Fireworks_right,Malin 1_right,Medusa Merger_right,Milky Way_right,Needle_right,Sculptor_right,Sombrero_right,Sunflower_right,Tadpole_right,Triangulum_right,Whirlpool_right,Home_Shots,Home_SoT,Home_Accurate passes,Home_Inaccurate passes,Home_Passes,Home_attacks,Home_chance,Away_Shots,Away_SoT,Away_Accurate passes,Away_Inaccurate passes,Away_Passes,Away_attacks,Away_chance
Game_ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
ID_KAG4KAE9,2017-02-15,1,1.0,Antennae,Andromeda,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8.0,2.0,265.0,75.0,340.0,37.0,0.0,5.0,1.0,433.0,75.0,508.0,34.0,0.0
ID_T7H1XN5Z,2016-09-14,1,12.0,Andromeda,Antennae,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5.0,1.0,354.0,89.0,443.0,54.0,0.0,10.0,0.0,386.0,76.0,462.0,60.0,0.0
ID_O1P9FGN4,2016-12-12,1,13.0,Andromeda,Butterfly,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3.0,0.0,554.0,115.0,669.0,25.0,0.0,6.0,2.0,279.0,109.0,388.0,17.0,0.0
ID_TR5M4KQV,2017-01-25,1,14.0,Andromeda,Cigar,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5.0,2.0,281.0,103.0,384.0,4.0,0.0,11.0,3.0,353.0,93.0,446.0,52.0,0.0
ID_EOIECRE2,2016-08-24,1,15.0,Andromeda,Circinus,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,13.0,2.0,436.0,106.0,542.0,46.0,0.0,5.0,0.0,452.0,114.0,566.0,36.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ID_PREVA6HZ,2016-08-28,1,59.0,Sculptor,Medusa Merger,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,4.0,0.0,447.0,109.0,556.0,34.0,0.0,10.0,3.0,347.0,87.0,434.0,48.0,0.0
ID_9DJ219JY,2017-02-19,1,151.0,Medusa Merger,Sculptor,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,15.0,5.0,413.0,90.0,503.0,42.0,0.0,9.0,1.0,338.0,72.0,410.0,51.0,0.0
ID_A1PN23J6,2016-10-23,1,154.0,Medusa Merger,Sombrero,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,14.0,4.0,365.0,72.0,437.0,82.0,0.0,12.0,2.0,263.0,83.0,346.0,38.0,0.0


In [ ]:
gs

,xt_value,Shots_Circinus,Shots_Backward,Shots_Sunflower,Shots_Coma Pinwheel,Shots_Eye of Sauron,Shots_Butterfly,Shots_Sombrero,Shots_Milky Way,Shots_Antennae,Shots_Whirlpool,Shots_Malin 1,Shots_Tadpole,Shots_Cosmos Redshift 7,Shots_Cigar,Shots_Cartwheel,Shots_Fireworks,Shots_Needle,Shots_Sculptor,Shots_Triangulum,Shots_Andromeda,Shots_Medusa Merger,Shots_Comet,SoT_Circinus,Accurate passes_Circinus,Inaccurate passes_Circinus,Passes_Circinus,SoT_Backward,Accurate passes_Backward,Inaccurate passes_Backward,Passes_Backward,SoT_Sunflower,Accurate passes_Sunflower,Inaccurate passes_Sunflower,Passes_Sunflower,SoT_Coma Pinwheel,Accurate passes_Coma Pinwheel,Inaccurate passes_Coma Pinwheel,Passes_Coma Pinwheel,SoT_Eye of Sauron,Accurate passes_Eye of Sauron,Inaccurate passes_Eye of Sauron,Passes_Eye of Sauron,SoT_Butterfly,Accurate passes_Butterfly,Inaccurate passes_Butterfly,Passes_Butterfly,SoT_Sombrero,Accurate passes_Sombrero,Inaccurate passes_Sombrero,Passes_Sombrero,SoT_Milky Way,Accurate passes_Milky Way,Inaccurate passes_Milky Way,Passes_Milky Way,SoT_Antennae,Accurate passes_Antennae,Inaccurate passes_Antennae,Passes_Antennae,SoT_Whirlpool,Accurate passes_Whirlpool,Inaccurate passes_Whirlpool,Passes_Whirlpool,SoT_Malin 1,Accurate passes_Malin 1,Inaccurate passes_Malin 1,Passes_Malin 1,SoT_Tadpole,Accurate passes_Tadpole,Inaccurate passes_Tadpole,Passes_Tadpole,SoT_Cosmos Redshift 7,Accurate passes_Cosmos Redshift 7,Inaccurate passes_Cosmos Redshift 7,Passes_Cosmos Redshift 7,SoT_Cigar,Accurate passes_Cigar,Inaccurate passes_Cigar,Passes_Cigar,SoT_Cartwheel,Accurate passes_Cartwheel,Inaccurate passes_Cartwheel,Passes_Cartwheel,SoT_Fireworks,Accurate passes_Fireworks,Inaccurate passes_Fireworks,Passes_Fireworks,SoT_Needle,Accurate passes_Needle,Inaccurate passes_Needle,Passes_Needle,SoT_Sculptor,Accurate passes_Sculptor,Inaccurate passes_Sculptor,Passes_Sculptor,SoT_Triangulum,Accurate passes_Triangulum,Inaccurate passes_Triangulum,Passes_Triangulum,SoT_Andromeda,Accurate passes_Andromeda,Inaccurate passes_Andromeda,Passes_Andromeda,SoT_Medusa Merger,Accurate passes_Medusa Merger,Inaccurate passes_Medusa Merger,Passes_Medusa Merger,SoT_Comet,Accurate passes_Comet,Inaccurate passes_Comet,Passes_Comet
Game_ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
ID_00EFNL7L,0.000000,0.0,0.0,0.0,0.0,0.0,19.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,327.0,83.0,410.0,0.0,0.0,0.0,0.0,0.0,263.0,86.0,349.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ID_00J124MI,0.000000,0.0,18.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.0,0.0,0.0,0.0,0.0,0.0,6.0,259.0,71.0,330.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,501.0,76.0,577.0,0.0,0.0,0.0,0.0
ID_03VRW8WV,2.754058,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,385.0,88.0,473.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,371.0,88.0,459.0
ID_04T9W310,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0

In [21]:
train['New_Date'] = np.nan

for id in train.index.tolist():
    train.loc[id, 'New_Date'] = int(train.loc[id, 'Date'].replace('-', ''))

test['New_Date'] = np.nan

for id in test.index.tolist():
    test.loc[id, 'New_Date'] = int(test.loc[id, 'Date'].replace('-', ''))

test

,Date,Season,Match_ID,Home Team,Away Team,Andromeda,Antennae,Backward,Butterfly,Cartwheel,Cigar,Circinus,Coma Pinwheel,Comet,Cosmos Redshift 7,Eye of Sauron,Fireworks,Malin 1,Medusa Merger,Milky Way,Needle,Sculptor,Sombrero,Sunflower,Tadpole,Triangulum,Whirlpool,Andromeda,Antennae,Backward,Butterfly,Cartwheel,Cigar,Circinus,Coma Pinwheel,Comet,Cosmos Redshift 7,Eye of Sauron,Fireworks,Malin 1,Medusa Merger,Milky Way,Needle,Sculptor,Sombrero,Sunflower,Tadpole,Triangulum,Whirlpool,Andromeda_right,Antennae_right,Backward_right,Butterfly_right,Cartwheel_right,Cigar_right,Circinus_right,Coma Pinwheel_right,Comet_right,Cosmos Redshift 7_right,Eye of Sauron_right,Fireworks_right,Malin 1_right,Medusa Merger_right,Milky Way_right,Needle_right,Sculptor_right,Sombrero_right,Sunflower_right,Tadpole_right,Triangulum_right,Whirlpool_right,Home_Shots,Home_SoT,Home_Accurate passes,Home_Inaccurate passes,Home_Passes,Home_attacks,Home_chance,Away_Shots,Away_SoT,Away_Accurate passes,Away_Inaccurate passes,Away_Passes,Away_attacks,Away_chance,New_Date
Game_ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
ID_8518U587,2018-12-08,3,102.0,Antennae,Cosmos Redshift 7,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,20.0,6.0,536.0,210.0,746.0,62.0,0.0,23.0,6.0,915.0,181.0,1096.0,90.0,2.0,20181208.0
ID_H49BIKG7,2018-09-11,3,18.0,Cosmos Redshift 7,Tadpole,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,6.0,2.0,335.0,72.0,407.0,36.0,1.0,8.0,2.0,281.0,84.0,365.0,33.0,0.0,20180911.0
ID_PO6SP4VA,2018-09-25,3,32.0,Cosmos Redshift 7,Medusa Merger,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,7.0,3.0,259.0,73.0,332.0,23.0,0.0,11.0,2.0,414.0,61.0,475.0,33.0,2.0,20180925.0
ID_MZRCNBAQ,2018-10-02,3,40.0,Cartwheel,Cosmos Redshift 7,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,9.0,0.0,447.0,76.0,523.0,40.0,0.0,8.0,4.0,343.0,78.0,421.0,14.0,2.0,20181002.0
ID_CV9VOLIU,2018-12-20,3,123.0,Cosmos Redshift 7,Needle,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,6.0,1.0,327.0,77.0,404.0,46.0,0.0,9.0,2.0,498.0,88.0,586.0,28.0,3.0,20181220.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ID_QKQ00O16,2018-12-08,3,300.0,Malin 1,Needle,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20181208.0
ID_172V2IXW,2018-12-11,3,300.0,Tadpole,Needle,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20181211.0
ID_2ZK5TE39,2018-12-22,3,300.0,Circinus,Malin 1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20181222.0


In [22]:
# train = train.iloc[:, :15]
'New_Date' in train.columns

True

In [ ]:
train.to_csv('new_train_4.csv')
test.to_csv('new_test_4.csv')

In [23]:
temp = pd.concat([train, target], axis=1)
corr = temp.corr().sort_values(['Score'], ascending=False)['Score']

In [24]:
corr.loc['Away_chance']

0.0221408995161552

In [25]:
import catboost as cb

def make_pool(X, y=None):
    return cb.Pool(X, y)

In [26]:
to_pop = ['Date', 'Home Team', 'Away Team']

# train = train.join(pd.get_dummies(train['Home Team']))
# train = train.join(pd.get_dummies(train['Away Team']), rsuffix='_away')
# test = test.join(pd.get_dummies(train['Home Team']))
# test = test.join(pd.get_dummies(train['Away Team']), rsuffix='_away')

for p in to_pop:
    train.pop(p)
    test.pop(p)

test

,Season,Match_ID,Andromeda,Antennae,Backward,Butterfly,Cartwheel,Cigar,Circinus,Coma Pinwheel,Comet,Cosmos Redshift 7,Eye of Sauron,Fireworks,Malin 1,Medusa Merger,Milky Way,Needle,Sculptor,Sombrero,Sunflower,Tadpole,Triangulum,Whirlpool,Andromeda,Antennae,Backward,Butterfly,Cartwheel,Cigar,Circinus,Coma Pinwheel,Comet,Cosmos Redshift 7,Eye of Sauron,Fireworks,Malin 1,Medusa Merger,Milky Way,Needle,Sculptor,Sombrero,Sunflower,Tadpole,Triangulum,Whirlpool,Andromeda_right,Antennae_right,Backward_right,Butterfly_right,Cartwheel_right,Cigar_right,Circinus_right,Coma Pinwheel_right,Comet_right,Cosmos Redshift 7_right,Eye of Sauron_right,Fireworks_right,Malin 1_right,Medusa Merger_right,Milky Way_right,Needle_right,Sculptor_right,Sombrero_right,Sunflower_right,Tadpole_right,Triangulum_right,Whirlpool_right,Home_Shots,Home_SoT,Home_Accurate passes,Home_Inaccurate passes,Home_Passes,Home_attacks,Home_chance,Away_Shots,Away_SoT,Away_Accurate passes,Away_Inaccurate passes,Away_Passes,Away_attacks,Away_chance,New_Date
Game_ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
ID_8518U587,3,102.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,20.0,6.0,536.0,210.0,746.0,62.0,0.0,23.0,6.0,915.0,181.0,1096.0,90.0,2.0,20181208.0
ID_H49BIKG7,3,18.0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,6.0,2.0,335.0,72.0,407.0,36.0,1.0,8.0,2.0,281.0,84.0,365.0,33.0,0.0,20180911.0
ID_PO6SP4VA,3,32.0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,7.0,3.0,259.0,73.0,332.0,23.0,0.0,11.0,2.0,414.0,61.0,475.0,33.0,2.0,20180925.0
ID_MZRCNBAQ,3,40.0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,9.0,0.0,447.0,76.0,523.0,40.0,0.0,8.0,4.0,343.0,78.0,421.0,14.0,2.0,20181002.0
ID_CV9VOLIU,3,123.0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,6.0,1.0,327.0,77.0,404.0,46.0,0.0,9.0,2.0,498.0,88.0,586.0,28.0,3.0,20181220.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ID_QKQ00O16,3,300.0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20181208.0
ID_172V2IXW,3,300.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20181211.0
ID_2ZK5TE39,3,300.0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20181222.0


In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 412 entries, ID_KAG4KAE9 to ID_VTVBHCO7
Columns: 350 entries, Season to New_Date
dtypes: float64(14), int64(1), uint8(335)
memory usage: 202.4+ KB


In [27]:
cols = pd.Series(train.columns)

for dup in cols[cols.duplicated()].unique(): 
    cols[cols[cols == dup].index.values.tolist()] = [dup + '_' + str(i) if i != 0 else dup for i in range(sum(cols == dup))]

# rename the columns with the cols list.
train.columns = cols
test.columns = cols


# Feature Permutation

In [35]:
from tqdm import tqdm


def generate_feats(fts, gen_ft=None, max_ft=100):
    numerics = ['float64', 'int64', 'uint8']
    corrs = fts.corr()
    if gen_ft is None:
        k = len(fts.columns)
        gen_ft = []
        for i in tqdm(range(k)):
            for j in range(i+1, k):
                try:
                    t1 = fts[fts.columns[i]].dtypes
                    t2 = fts[fts.columns[j]].dtypes
                    if t1 not in numerics or t2 not in numerics:
                        continue
                    if corrs[fts.columns[i]].iloc[j] != np.nan and (corrs[fts.columns[i]].iloc[j] >= 0.5):
                        gen_ft.append([i, j])
                except:
                    pass
    for q in tqdm(range(min(max_ft, len(gen_ft)))):
        i, j = gen_ft[q]
        fts[f"{fts.columns[i]}*{fts.columns[j]}"] = fts[fts.columns[i]] * fts[fts.columns[j]]
        fts[f"{fts.columns[i]}/{fts.columns[j]}"] = fts[fts.columns[i]] / (fts[fts.columns[j]]+1e-9)
        fts[f"{fts.columns[j]}/{fts.columns[i]}"] = fts[fts.columns[j]] / (fts[fts.columns[i]]+1e-9)
        fts[f"{fts.columns[i]}_func1_{fts.columns[j]}"] = fts[fts.columns[i]] * fts[fts.columns[j]] / (fts[fts.columns[i]] + fts[fts.columns[j]] + 1e-9) ** 2
        fts[f"{fts.columns[i]}_func2_{fts.columns[j]}"] = abs(fts[fts.columns[i]] - fts[fts.columns[j]]) ** 0.5 * abs(fts[fts.columns[i]] + fts[fts.columns[j]]) ** 0.5
    
    # fts = reduce_mem_usage(fts)
    return fts, gen_ft

In [36]:
df = pd.concat([train, test], axis=0)
fts, gen_ft = generate_feats(df, max_ft=100)
fts

  0%|          | 0/42 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:25: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:26: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) ins

,Season,Match_ID,Andromeda,Antennae,Backward,Butterfly,Cartwheel,Cigar,Circinus,Coma Pinwheel,Comet,Cosmos Redshift 7,Eye of Sauron,Fireworks,Malin 1,Medusa Merger,Milky Way,Needle,Sculptor,Sombrero,Sunflower,Tadpole,Triangulum,Whirlpool,Andromeda_1,Antennae_1,Backward_1,Butterfly_1,Cartwheel_1,Cigar_1,Circinus_1,Coma Pinwheel_1,Comet_1,Cosmos Redshift 7_1,Eye of Sauron_1,Fireworks_1,Malin 1_1,Medusa Merger_1,Milky Way_1,Needle_1,Sculptor_1,Sombrero_1,Sunflower_1,Tadpole_1,Triangulum_1,Whirlpool_1,Andromeda_right,Antennae_right,Backward_right,Butterfly_right,Cartwheel_right,Cigar_right,Circinus_right,Coma Pinwheel_right,Comet_right,Cosmos Redshift 7_right,Eye of Sauron_right,Fireworks_right,Malin 1_right,Medusa Merger_right,Milky Way_right,Needle_right,Sculptor_right,Sombrero_right,Sunflower_right,Tadpole_right,Triangulum_right,Whirlpool_right,Home_Shots,Home_SoT,Home_Accurate passes,Home_Inaccurate passes,Home_Passes,Home_attacks,Home_chance,Away_Shots,Away_SoT,Away_Accurate passes,Away_Inaccurate passes,Away_Passes,Away_attacks,Away_chance,New_Date,Season*New_Date,Season/New_Date,New_Date/Season,Season_func1_New_Date,Season_func2_New_Date,Andromeda_1*Andromeda_right,Andromeda_1/Andromeda_right,Andromeda_right/Andromeda_1,Andromeda_1_func1_Andromeda_right,Andromeda_1_func2_Andromeda_right,Antennae_1*Antennae_right,Antennae_1/Antennae_right,Antennae_right/Antennae_1,Antennae_1_func1_Antennae_right,Antennae_1_func2_Antennae_right,Backward_1*Backward_right,Backward_1/Backward_right,Backward_right/Backward_1,Backward_1_func1_Backward_right,Backward_1_func2_Backward_right,Butterfly_1*Butterfly_right,Butterfly_1/Butterfly_right,Butterfly_right/Butterfly_1,Butterfly_1_func1_Butterfly_right,Butterfly_1_func2_Butterfly_right,Cartwheel_1*Cartwheel_right,Cartwheel_1/Cartwheel_right,Cartwheel_right/Cartwheel_1,Cartwheel_1_func1_Cartwheel_right,Cartwheel_1_func2_Cartwheel_right,Cigar_1*Cigar_right,Cigar_1/Cigar_right,Cigar_right/Cigar_1,Cigar_1_func1_Cigar_right,Cigar_1_func2_Cigar_right,Circinus_1*Circinus_right,Circinus_1/Circinus_right,Circinus_right/Circinus_1,Circinus_1_func1_Circinus_right,Circinus_1_func2_Circinus_right,Coma Pinwheel_1*Coma Pinwheel_right,Coma Pinwheel_1/Coma Pinwheel_right,Coma Pinwheel_right/Coma Pinwheel_1,Coma Pinwheel_1_func1_Coma Pinwheel_right,Coma Pinwheel_1_func2_Coma Pinwheel_right,Comet_1*Comet_right,Comet_1/Comet_right,Comet_right/Comet_1,Comet_1_func1_Comet_right,Comet_1_func2_Comet_right,Cosmos Redshift 7_1*Cosmos Redshift 7_right,Cosmos Redshift 7_1/Cosmos Redshift 7_right,Cosmos Redshift 7_right/Cosmos Redshift 7_1,Cosmos Redshift 7_1_func1_Cosmos Redshift 7_right,Cosmos Redshift 7_1_func2_Cosmos Redshift 7_right,Eye of Sauron_1*Eye of Sauron_right,Eye of Sauron_1/Eye of Sauron_right,Eye of Sauron_right/Eye of Sauron_1,Eye of Sauron_1_func1_Eye of Sauron_right,Eye of Sauron_1_func2_Eye of Sauron_right,Fireworks_1*Fireworks_right,Fireworks_1/Fireworks_right,Fireworks_right/Fireworks_1,Fireworks_1_func1_Fireworks_right,Fireworks_1_func2_Fireworks_right,Malin 1_1*Malin 1_right,Malin 1_1/Malin 1_right,Malin 1_right/Malin 1_1,Malin 1_1_func1_Malin 1_right,Malin 1_1_func2_Malin 1_right,Medusa Merger_1*Medusa Merger_right,Medusa Merger_1/Medusa Merger_right,Medusa Merger_right/Medusa Merger_1,Medusa Merger_1_func1_Medusa Merger_right,Medusa Merger_1_func2_Medusa Merger_right,Milky Way_1*Milky Way_right,Milky Way_1/Milky Way_right,Milky Way_right/Milky Way_1,Milky Way_1_func1_Milky Way_right,Milky Way_1_func2_Milky Way_right,Needle_1*Needle_right,Needle_1/Needle_right,Needle_right/Needle_1,Needle_1_func1_Needle_right,Needle_1_func2_Needle_right,Sculptor_1*Sculptor_right,Sculptor_1/Sculptor_right,Sculptor_right/Sculptor_1,Sculptor_1_func1_Sculptor_right,Sculptor_1_func2_Sculptor_right,Sombrero_1*Sombrero_right,Sombrero_1/Sombrero_right,Sombrero_right/Sombrero_1,Sombrero_1_func1_Sombrero_right,Sombrero_1_func2_Sombrero_right,Sunflower_1*Sunflower_right,Sunflower_1/Sunflower_right,Sunflower_right/Sunflower_1,S

In [ ]:
# newtrain = fts.loc[train_ids]
# newtest = fts.loc[test_ids]
# newtest

,Season,Match_ID,Andromeda,Antennae,Backward,Butterfly,Cartwheel,Cigar,Circinus,Coma Pinwheel,Comet,Cosmos Redshift 7,Eye of Sauron,Fireworks,Malin 1,Medusa Merger,Milky Way,Needle,Sculptor,Sombrero,Sunflower,Tadpole,Triangulum,Whirlpool,Andromeda_1,Antennae_1,Backward_1,Butterfly_1,Cartwheel_1,Cigar_1,Circinus_1,Coma Pinwheel_1,Comet_1,Cosmos Redshift 7_1,Eye of Sauron_1,Fireworks_1,Malin 1_1,Medusa Merger_1,Milky Way_1,Needle_1,Sculptor_1,Sombrero_1,Sunflower_1,Tadpole_1,Triangulum_1,Whirlpool_1,Andromeda_right,Antennae_right,Backward_right,Butterfly_right,Cartwheel_right,Cigar_right,Circinus_right,Coma Pinwheel_right,Comet_right,Cosmos Redshift 7_right,Eye of Sauron_right,Fireworks_right,Malin 1_right,Medusa Merger_right,Milky Way_right,Needle_right,Sculptor_right,Sombrero_right,Sunflower_right,Tadpole_right,Triangulum_right,Whirlpool_right,2016-08-03,2016-08-04,2016-08-06,2016-08-09,2016-08-10,2016-08-11,2016-08-14,2016-08-20,2016-08-24,2016-08-25,2016-08-27,2016-08-28,2016-09-14,2016-09-15,2016-09-21,2016-09-22,2016-09-24,2016-09-27,2016-09-28,2016-09-29,2016-10-01,2016-10-02,2016-10-23,2016-10-25,2016-10-26,2016-10-27,2016-10-29,2016-10-30,2016-11-05,2016-11-06,2016-11-08,2016-11-09,2016-11-10,2016-11-26,2016-11-27,2016-11-30,2016-12-01,2016-12-06,2016-12-07,2016-12-08,2016-12-11,2016-12-12,2016-12-17,2016-12-20,2016-12-21,2016-12-22,2017-01-03,2017-01-04,2017-01-05,2017-01-07,2017-01-08,2017-01-11,2017-01-12,2017-01-15,2017-01-17,2017-01-18,2017-01-19,2017-01-24,2017-01-25,2017-01-26,2017-01-31,2017-02-01,2017-02-02,2017-02-12,2017-02-14,2017-02-15,2017-02-16,2017-02-19,2017-02-28,2017-03-01,2017-03-06,2017-03-07,2017-03-08,2017-03-10,2017-03-11,2017-08-11,2017-08-12,2017-08-14,2017-08-15,2017-08-16,2017-08-17,2017-08-18,2017-08-19,2017-08-21,2017-08-22,2017-08-23,2017-08-24,2017-08-25,2017-08-27,2017-08-28,2017-08-29,2017-08-30,2017-09-02,2017-09-05,2017-10-24,2017-10-25,2017-10-27,2017-10-28,2017-11-03,2017-11-04,2017-11-21,2017-11-22,2017-11-24,2017-11-25,2017-11-27,2017-11-28,2017-11-29,2017-12-05,2017-12-06,2017-12-09,2017-12-15,2017-12-16,2017-12-18,2017-12-19,2017-12-20,2018-01-02,2018-01-05,2018-01-06,2018-01-08,2018-01-09,2018-01-10,2018-01-13,2018-01-16,2018-01-17,2018-01-19,2018-01-20,2018-01-23,2018-01-24,2018-01-26,2018-01-27,2018-01-29,2018-01-30,2018-01-31,2018-02-02,2018-02-09,2018-02-12,2018-02-13,2018-02-14,2018-02-17,2018-02-20,2018-02-21,2018-02-24,2018-03-02,2018-03-03,2018-03-05,2018-03-06,2018-03-07,2018-03-10,2018-03-13,2018-03-19,2018-03-20,2018-03-21,2018-04-03,2018-04-04,2018-04-06,2018-04-09,2018-04-10,2018-04-11,2018-04-12,2018-04-18,2018-04-20,2018-04-21,2018-04-23,2018-04-24,2018-04-25,2018-04-27,2018-04-28,2018-04-30,2018-05-01,2018-05-02,2018-05-04,2018-05-05,2018-05-06,2018-05-09,2018-05-11,2018-05-12,2018-05-15,2018-05-18,2018-05-19,2018-05-26,2018-05-29,2018-06-02,2018-06-05,2018-08-21,2018-08-22,2018-08-24,2018-08-25,2018-09-11,2018-09-12,2018-09-14,2018-09-18,2018-09-19,2018-09-22,2018-09-24,2018-09-25,2018-09-26,2018-10-02,2018-10-03,2018-10-16,2018-10-17,2018-10-19,2018-10-20,2018-10-23,2018-10-24,2018-10-27,2018-10-28,2018-11-02,2018-11-03,2018-11-04,2018-11-05,2018-11-06,2018-11-07,2018-11-20,2018-11-21,2018-11-24,2018-11-27,2018-11-28,2018-12-01,2018-12-02,2018-12-04,2018-12-05,2018-12-07,2018-12-08,2018-12-11,2018-12-12,2018-12-14,2018-12-17,2018-12-18,2018-12-19,2018-12-20,2018-12-22,2018-12-23,2019-02-07,2019-02-14,2019-02-15,2019-02-16,2019-02-19,2019-02-20,2019-02-22,2019-02-25,2019-02-26,2019-02-27,2019-03-02,2019-03-05,2019-03-06,2019-03-16,2019-03-19,2019-03-20,2019-04-02,2019-04-03,2019-04-05,2019-04-06,2019-04-12,2019-04-16,2019-04-17,2019-04-23,2019-04-24,2019-04-27,2019-04-30,2019-05-02,2019-05-03,2019-05-04,2019-05-06,2019-05-07,2019-05-08,2019-05-14,2019-05-21,2019-05-23,2019-05-24,Home_Shots,Home_SoT,Home_Accurate passes,Home_Inaccurate passes,Home_Passes,Home_attacks,Away_Shots,Away_SoT,Away_Accurate passes,Away_Inaccurate passes,Away_Passes,Away_attacks,New_Dat

# Feature Threshold

In [37]:
from sklearn.model_selection import train_test_split

SEED = 2417
X_train, X_val, y_train, y_val = train_test_split(train, target, test_size=0.2, random_state=SEED)
# X_train, X_val, y_train, y_val = train_test_split(newtrain, target, test_size=0.2, random_state=SEED)

In [38]:
models = []
good_ft = None

for i in range(1):
    model = cb.CatBoostClassifier(iterations=200,
                                  #max_depth=6,
                                  #l2_leaf_reg=3.2,
                                  loss_function='MultiClass', 
                                  random_seed=SEED)
    
    pool_train, pool_val = make_pool(X_train, y_train), make_pool(X_val, y_val)
    
    model.fit(pool_train, eval_set=pool_val, plot=False, verbose=100)
    ftt = model.get_feature_importance(prettified=True)
    good_ft = ftt['Feature Id'][ftt['Importances'] >= 1]
    break

good_ft

Learning rate set to 0.195289
0:	learn: 1.0805852	test: 1.1009972	best: 1.1009972 (0)	total: 81.3ms	remaining: 16.2s
199:	learn: 0.1385407	test: 1.0545871	best: 1.0045255 (82)	total: 1.01s	remaining: 0us

bestTest = 1.00452549
bestIteration = 82

Shrink model to first 83 iterations.


0     Away_Inaccurate passes
1               Home_attacks
2                   Match_ID
3     Home_Inaccurate passes
4       Away_Accurate passes
5               Away_attacks
6                Away_Passes
7       Home_Accurate passes
8                   New_Date
9                Home_Passes
10                Home_Shots
11                  Away_SoT
12                Away_Shots
13                  Home_SoT
14       Medusa Merger_right
15             Medusa Merger
16                    Season
17                 Fireworks
18                  Backward
Name: Feature Id, dtype: object

# Training

In [40]:
seeds = [2417, 1724, 24, 17]

models = []
good_ft = train.columns.tolist()
# pool_train, pool_val = make_pool(X_train, y_train), make_pool(X_val, y_val)
pool_train, pool_val = make_pool(X_train[good_ft], y_train), make_pool(X_val[good_ft], y_val)

for sd in seeds:
    model = cb.CatBoostClassifier(iterations=1000,
                                  # max_depth=8,
                                  # learning_rate=0.1,
                                  loss_function='MultiClass', 
                                  random_seed=sd)

    model.fit(pool_train, eval_set=pool_val, plot=False, verbose=100)
    models.append(model)

Learning rate set to 0.105602
0:	learn: 1.0885815	test: 1.0996522	best: 1.0996522 (0)	total: 7.51ms	remaining: 7.51s
100:	learn: 0.5161123	test: 1.0494105	best: 1.0447989 (42)	total: 491ms	remaining: 4.37s
200:	learn: 0.2784153	test: 1.0441047	best: 1.0393192 (193)	total: 947ms	remaining: 3.77s
300:	learn: 0.1754068	test: 1.0576952	best: 1.0382330 (227)	total: 1.42s	remaining: 3.3s
400:	learn: 0.1252648	test: 1.0636643	best: 1.0382330 (227)	total: 1.9s	remaining: 2.84s
500:	learn: 0.0920300	test: 1.0652044	best: 1.0382330 (227)	total: 2.37s	remaining: 2.36s
600:	learn: 0.0709844	test: 1.0770090	best: 1.0382330 (227)	total: 2.82s	remaining: 1.87s
700:	learn: 0.0567651	test: 1.0926398	best: 1.0382330 (227)	total: 3.29s	remaining: 1.41s
800:	learn: 0.0474482	test: 1.1061316	best: 1.0382330 (227)	total: 3.74s	remaining: 929ms
900:	learn: 0.0401655	test: 1.1246268	best: 1.0382330 (227)	total: 4.21s	remaining: 463ms
999:	learn: 0.0348533	test: 1.1371441	best: 1.0382330 (227)	total: 4.7s	rema

In [41]:
test

,Season,Match_ID,Andromeda,Antennae,Backward,Butterfly,Cartwheel,Cigar,Circinus,Coma Pinwheel,Comet,Cosmos Redshift 7,Eye of Sauron,Fireworks,Malin 1,Medusa Merger,Milky Way,Needle,Sculptor,Sombrero,Sunflower,Tadpole,Triangulum,Whirlpool,Andromeda_1,Antennae_1,Backward_1,Butterfly_1,Cartwheel_1,Cigar_1,Circinus_1,Coma Pinwheel_1,Comet_1,Cosmos Redshift 7_1,Eye of Sauron_1,Fireworks_1,Malin 1_1,Medusa Merger_1,Milky Way_1,Needle_1,Sculptor_1,Sombrero_1,Sunflower_1,Tadpole_1,Triangulum_1,Whirlpool_1,Andromeda_right,Antennae_right,Backward_right,Butterfly_right,Cartwheel_right,Cigar_right,Circinus_right,Coma Pinwheel_right,Comet_right,Cosmos Redshift 7_right,Eye of Sauron_right,Fireworks_right,Malin 1_right,Medusa Merger_right,Milky Way_right,Needle_right,Sculptor_right,Sombrero_right,Sunflower_right,Tadpole_right,Triangulum_right,Whirlpool_right,Home_Shots,Home_SoT,Home_Accurate passes,Home_Inaccurate passes,Home_Passes,Home_attacks,Home_chance,Away_Shots,Away_SoT,Away_Accurate passes,Away_Inaccurate passes,Away_Passes,Away_attacks,Away_chance,New_Date
Game_ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
ID_8518U587,3,102.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,20.0,6.0,536.0,210.0,746.0,62.0,0.0,23.0,6.0,915.0,181.0,1096.0,90.0,2.0,20181208.0
ID_H49BIKG7,3,18.0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,6.0,2.0,335.0,72.0,407.0,36.0,1.0,8.0,2.0,281.0,84.0,365.0,33.0,0.0,20180911.0
ID_PO6SP4VA,3,32.0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,7.0,3.0,259.0,73.0,332.0,23.0,0.0,11.0,2.0,414.0,61.0,475.0,33.0,2.0,20180925.0
ID_MZRCNBAQ,3,40.0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,9.0,0.0,447.0,76.0,523.0,40.0,0.0,8.0,4.0,343.0,78.0,421.0,14.0,2.0,20181002.0
ID_CV9VOLIU,3,123.0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,6.0,1.0,327.0,77.0,404.0,46.0,0.0,9.0,2.0,498.0,88.0,586.0,28.0,3.0,20181220.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ID_QKQ00O16,3,300.0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20181208.0
ID_172V2IXW,3,300.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20181211.0
ID_2ZK5TE39,3,300.0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20181222.0


In [ ]:
# seeds = [655, 56, 42, 10, 4, 0]

# for sd in seeds:
#     model = cb.CatBoostClassifier(iterations=500,
#                                   # max_depth=8,
#                                   loss_function='MultiClass', 
#                                   random_seed=sd)

#     model.fit(pool_train, eval_set=pool_val, plot=False, verbose=100)
#     models.append(model)

In [42]:
def predict_mass(X, models):
    sub = pd.Series(index=X.index)
    for id in sub.index.tolist():
        print(id)
        if X.loc[id, 'Home_chance'] > X.loc[id, 'Away_chance']:
            sub.loc[id] = 'Home Win'
        elif X.loc[id, 'Home_chance'] < X.loc[id, 'Away_chance']:
            sub.loc[id] = 'Away Win'
        else:
            sub.loc[id] = 'Draw'
    sub = pd.get_dummies(sub).rename(columns={'Away Win': 0, 'Draw': 1, 'Home Win': 2}).astype('float64')
    print(sub)
    pr = [model.predict_proba(X) for model in models]
    pr.append(sub)
    preds = pd.DataFrame(np.array(np.mean(pr, axis=0)))
    print(0)
    # preds = pd.DataFrame(np.array(preds).transpose(1, 0), index=X.index, columns=[f'type_{i}' for i in range(35)]).astype(float)
    return preds

def pred(X, models):
    preds = predict_mass(X[good_ft], models)
    # print(preds)
    preds.index = X.index
    # preds = pd.Series(preds.iloc[:, 0])
    # preds.replace({0: "Away Win", 1: "Draw", 2: "Home Win"}, inplace=True)
    preds.rename({0: "Away Win", 1: "Draw", 2: "Home Win"}, inplace=True)

    # print(preds)
    #               0: "Low Cost", 1: "Lower Cost", 2: "Normal Cost"}, inplace=True)
    # preds = preds[['High Cost', 'Higher Cost', 'Highest Cost', 'Low Cost', 'Lower Cost', 'Normal Cost']]
    return preds

In [43]:
from sklearn.metrics import accuracy_score

y_val_text = y_val.replace({0: "Away Win", 1: "Draw", 2: "Home Win"})
# y_val_ohe = y_val_ohe[['High Cost', 'Higher Cost', 'Highest Cost', 'Low Cost', 'Lower Cost', 'Normal Cost']]
preds = pred(X_val, models)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:163: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = asanyarray(a)


ID_OQM43U7K
ID_9JEI8DWV
ID_0A2WEVY3
ID_SXSLZSI1
ID_XFQO60PZ
ID_RAT7AK8R
ID_BOI7CXJV
ID_WX90Q0Q3
ID_MWRHCSMW
ID_HB9WSKP9
ID_UATYFRPS
ID_36NZ9IU5
ID_27YV4RRO
ID_SREWU5C6
ID_XMM4Z5EZ
ID_YXXCNK2G
ID_KST7PRMU
ID_WZASJ4IV
ID_VVG172LB
ID_ZWPPN4W1
ID_XWI57Q9K
ID_UMASWHEF
ID_CT02DQO5
ID_MWCIADS6
ID_V891OYVP
ID_KYEZ4Q5U
ID_MUYK98UO
ID_KAHZNQZ4
ID_JL0A7NYU
ID_T31AU81S
ID_UXUK6H9F
ID_72XLLS9T
ID_8ITGS2P3
ID_C1EA08ZR
ID_O7S3MLNP
ID_IDJ1L6GL
ID_24HKPKQH
ID_UGM73VHI
ID_XEI81LPI
ID_R3I7GSLU
ID_A2FD42KH
ID_NO60D4TO
ID_KSFJ2RGK
ID_68FWDT6H
ID_ULFMIS9R
ID_LEBJ75LN
ID_WDA74NYV
ID_PMX7KBOK
ID_QMJBCJTO
ID_UC7PMJ25
ID_DF19AH9G
ID_9KVVOCR6
ID_QAJGGM73
ID_S2DH4W65
ID_NEG8CDCD
ID_R6C3GV9H
ID_T62C8AP0
ID_5KQZOTHQ
ID_BJ2VX0EX
ID_MI4W134C
ID_GOIGEZE8
ID_L76JBE84
ID_Z71PIKP3
ID_MWHDDOC0
ID_GUGKLLI9
ID_22R4A4OV
ID_06QXUEYD
ID_OC87DVM9
ID_0PBGEMAI
ID_DPPFAJ6O
ID_L3GUMYJ4
ID_JW43FR1J
ID_THU0JYSJ
ID_7YEZ3VK0
ID_OZKQYKQF
ID_BJHEDY9T
ID_2947LQTO
ID_KKRFBDM9
ID_5XNJBBBU
ID_JP06JI0C
ID_T12UUJG7
ID_UKIMWWUK
ID_ZE7D4JAP
    

ValueError: ignored

In [44]:
preds

NameError: ignored

In [ ]:
preds

,0,1,2
Game_ID,,,
ID_OQM43U7K,0.495686,0.302534,0.201780
ID_9JEI8DWV,0.384678,0.312922,0.302400
ID_0A2WEVY3,0.155108,0.091769,0.753124
ID_SXSLZSI1,0.319172,0.368618,0.312210
ID_XFQO60PZ,0.270765,0.254332,0.474903
...,...,...,...
ID_5XNJBBBU,0.060234,0.150768,0.788998
ID_JP06JI0C,0.081223,0.315147,0.603630
ID_T12UUJG7,0.165823,0.280769,0.553408


In [ ]:
y_val_text

Game_ID
ID_OQM43U7K    Away Win
ID_9JEI8DWV        Draw
ID_0A2WEVY3    Away Win
ID_SXSLZSI1    Away Win
ID_XFQO60PZ        Draw
                 ...   
ID_5KQZOTHQ    Away Win
ID_BJ2VX0EX    Home Win
ID_MI4W134C        Draw
ID_GOIGEZE8    Home Win
ID_L76JBE84    Away Win
Name: Score, Length: 62, dtype: object

In [45]:
preds = pred(test, models[:-1])
preds

ID_8518U587
ID_H49BIKG7
ID_PO6SP4VA
ID_MZRCNBAQ
ID_CV9VOLIU
ID_EP02V6O9
ID_D1BKOMNB
ID_GGFY5Z38
ID_YQ6DCSJJ
ID_ZIMAODWU
ID_H6V0DHCM
ID_EFUM1IB3
ID_7H9RUYR2
ID_7JTHUMTF
ID_VZDE4BA3
ID_1Y735BT9
ID_C51DS9XE
ID_EEAWBOYL
ID_OKHY3GBQ
ID_JKJP61NL
ID_ZRZC49M1
ID_NQGKKQRG
ID_KL74YZ1W
ID_D4BBONMS
ID_E4NAVOHG
ID_8AKT073X
ID_SGHWIP1E
ID_OHS6RR07
ID_0BFR12C0
ID_O3EQ8WEA
ID_PD70LTRE
ID_B9JSESNH
ID_7ZQ7E7Q2
ID_S2ZLOYQU
ID_UUQYAGBP
ID_2BHM1MAF
ID_2B96PJLU
ID_QKWD8147
ID_NMRV6S78
ID_NQX0L8NJ
ID_GGRLN92Z
ID_8LEIJZJI
ID_H4WRH27A
ID_TRN20YF5
ID_313CQJ6L
ID_60EADYBI
ID_4Q60CD7W
ID_LWI3010X
ID_N4FNUPW8
ID_G91KCS98
ID_RFU5FMC8
ID_GBP5RWBW
ID_EWFWK6L1
ID_QCQ20X8Y
ID_03VRW8WV
ID_XQGYMP2Q
ID_A46DVQWP
ID_0UB8ADG9
ID_FDCEHVOD
ID_SY9KA6L3
ID_RX2CNZIS
ID_B3NVY9HP
ID_MXDJNYS9
ID_WJIU8X3N
ID_DYAVEIYE
ID_L80OJ6QI
ID_A5U63MGQ
ID_B5ED50XX
ID_2X3AC0BS
ID_ZTDZNU6I
ID_B32VWKSX
ID_D8RLSTH5
ID_7RVSY2WQ
ID_DLPZZFIJ
ID_IPJJKELT
ID_MM9562CN
ID_ZJC5BV5Z
ID_HNY0SOAW
ID_RPNHBU1X
ID_L1OMXXIV
ID_QPA4T2QH
ID_VP9Q1486
ID_PK7PTLYZ
ID_E

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  


,0,1,2
Game_ID,,,
ID_8518U587,0.546079,0.070126,0.383795
ID_H49BIKG7,0.143240,0.416602,0.440158
ID_PO6SP4VA,0.647053,0.283582,0.069365
ID_MZRCNBAQ,0.493171,0.197743,0.309085
ID_CV9VOLIU,0.303640,0.345138,0.351222
...,...,...,...
ID_QKQ00O16,0.294609,0.592398,0.112992
ID_172V2IXW,0.242051,0.639743,0.118206
ID_2ZK5TE39,0.302799,0.593007,0.104194


In [46]:
preds.loc['ID_8518U587']

0    0.546079
1    0.070126
2    0.383795
Name: ID_8518U587, dtype: float64

In [ ]:
preds.loc['ID_8518U587']

0    0.505382
1    0.096500
2    0.398118
Name: ID_8518U587, dtype: float64

In [47]:
pr2 = pd.Series(index=preds.index)
for id in preds.index.tolist():
    # print(id)
    val = preds.loc[id].max()
    if preds.loc[id, 0] == val:
        preds.loc[id, 0] = 1
        preds.loc[id, 1] = 0
        preds.loc[id, 2] = 0
        pr2[id] = 0
    elif preds.loc[id, 1] == val:
        preds.loc[id, 0] = 0
        preds.loc[id, 1] = 1
        preds.loc[id, 2] = 0
        pr2[id] = 1
    elif preds.loc[id, 2] == val:
        preds.loc[id, 0] = 0
        preds.loc[id, 1] = 0
        preds.loc[id, 2] = 1
        pr2[id] = 2

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  """Entry point for launching an IPython kernel.


In [48]:
pr2.replace({0: "Away Win", 1: "Draw", 2: "Home Win"}, inplace=True)

In [49]:
pr2

Game_ID
ID_8518U587    Away Win
ID_H49BIKG7    Home Win
ID_PO6SP4VA    Away Win
ID_MZRCNBAQ    Away Win
ID_CV9VOLIU    Home Win
                 ...   
ID_QKQ00O16        Draw
ID_172V2IXW        Draw
ID_2ZK5TE39        Draw
ID_IS54QGW9        Draw
ID_YG9IONLS        Draw
Length: 234, dtype: object

In [ ]:
pr2.to_csv('football22.csv')